# Overview
This notebook identifies significant r values
* fisherZ correct
* stack across participants and runs
* run t-test and identify r values greater than 0
* reconvert fisherZ back to correlation coefficients
* stack coefficients across hemisphere for multiple comparisons
* save results in corresponding hemisphere while preserving medial/non-medial wall vertices

# TODO Aug/2024
* comb-r2
* Need to grab comb-r2 instead of agents-r or bg-r.

### modules

In [ ]:
import numpy as np
import pandas as pd
import os
from os.path import join
import scipy.stats
from statsmodels.stats.multitest import multipletests
import nibabel as nib
from pathlib import Path

### function: write_gifti

In [ ]:
def write_gifti_v2(data, output_fn, template_fn):
    gii = nib.load(template_fn)
    for i in np.arange(gii.numDA):
        gii.remove_gifti_data_array(0)
    gda = nib.gifti.GiftiDataArray(data)
    gii.add_gifti_data_array(gda)
    # nib.gifti.write(gii, output_fn)
    gii.to_filename(output_fn)

# parameters for Nested "actions bg moten" -- isolate "agents"
TODO: 
1. load comb-r2 for full model & comb-r2 for glove_nested_actions-bg-moten -> 
   * uniq = (comb_r2_full - comb_r2_nested) / comb_r2_full
2. Do this iteratively for all nested models (total of 3)
3. t-tests of uniq against 0. 
4. brain map: masked unique proportion R2, based on fdr-corrected t values. (Expected to be similar to Figure 1. )

In [ ]:
suma_dir = '/Users/h/suma-fsaverage6'
main_dir = '/dartfs/rc/lab/D/DBIC/DBIC/f0042x1/life-encoding'
main_dir = '/Volumes/life-encoding'
features = 'actionsmoten'

ANALYSIS="glove_nested_actions-bg-moten" # agents
# "glove_nested_agents-bg-moten"
# "glove_nested_actions-agents-moten"

output_dir = os.path.join(main_dir, 'results', 'himalaya', 'glove_nested', 'actions-bg-moten', 'ha_common_pca-40')
alignment = 'ha_common'
results = ['comb-r2']#,  'moten-r']
runs = [1, 2, 3, 4]
hemis = ['lh', 'rh']
pca_comp = 40
fmri_durs = {1: 374, 2: 346, 3: 377, 4: 412}
n_samples = 1509
n_vertices = 40962
n_medial = {'lh': 3486, 'rh': 3491}
subjects = ['sub-rid000001', 'sub-rid000005', 'sub-rid000006',
            'sub-rid000009', 'sub-rid000012', 'sub-rid000014',
            'sub-rid000017', 'sub-rid000019', 'sub-rid000024',
            'sub-rid000027', 'sub-rid000031', 'sub-rid000032',
            'sub-rid000033', 'sub-rid000034', 'sub-rid000036',
            'sub-rid000037', 'sub-rid000038', 'sub-rid000041']



# compute one sample t-tests

In [ ]:
for result in results:
    hemi_t = []
    hemi_p = []
    hemi_mean = []
    print(f"starting {result} ________________")
    for hemisphere in hemis:
        medial_mask = np.load(os.path.join(main_dir, 'data', f'fsaverage6_medial_{hemisphere}.npy'))
        assert np.sum(medial_mask) == n_medial[hemisphere]
        cortical_vertices = ~medial_mask # boolean (true for non-medials, false for medials)
        cortical_coords = np.where(cortical_vertices)[0] # returns indices of non-medials
        avg_all = []
        # NOTE 1. Fisher z transform per run per participant (np.arctanh)
        for test_subject in subjects:
            stack_fisherz_run = []
            for test_run in runs:           

                run_data = np.load(f"{output_dir}/{result}_pca-{pca_comp}_align-{alignment}_{test_subject}_run-{test_run}_hemi-{hemisphere}.npy")
                fisherz_run = np.arctanh(run_data[0, cortical_vertices])
                stack_fisherz_run.append(fisherz_run)
        
            # NOTE: 2. average (z-transformed) correlations across runs: yields 18 maps (1 per subject)
            avg_run = np.mean(np.vstack(stack_fisherz_run), axis = 0)
            avg_all.append(avg_run)
        fisherz_all = np.vstack(avg_all) # 18 x nonmedial (per hemi)

        # NOTE: 3. Scipy ttest_1samp to get t-value and p-value
        t, p = scipy.stats.ttest_1samp(fisherz_all, popmean=0,axis=0, alternative='greater')
        hemi_t.append(t)
        hemi_p.append(p)
        hemi_mean.append(np.tanh(np.nanmean(fisherz_all, axis = 0)))

    # %% NOTE: 4-1. concatenate (np.hstack) the two hemispheres p-values (and exclude medial wall) prior to computing FDR (load in cortical_vertices.npy)
    left_vert = hemi_p[0].shape[0]
    t_all = np.hstack(hemi_t)
    p_all = np.hstack(hemi_p)

    reject, q_all, _, _ = multipletests(p_all, method = 'fdr_bh')
    q_both = [q_all[:left_vert], q_all[left_vert:]]
    print("* completed t-tests for both hemispheres")
    hemilabels = ['lh', 'rh']
    Path(join(output_dir, 'stats')).mkdir(parents = True, exist_ok = True)
    for h, hemisphere in enumerate(hemis):
        stats = np.zeros((n_vertices))
        hemi_mean[h][q_both[h] >= .05] = 0
        medial_mask = np.load(os.path.join(main_dir, 'data', f'fsaverage6_medial_{hemisphere}.npy'))
        assert np.sum(medial_mask) == n_medial[hemisphere]
        cortical_vertices = ~medial_mask # boolean (true for non-medials, false for medials)
        cortical_coords = np.where(cortical_vertices)[0] # returns indices of non-medials
        stats[cortical_coords] = hemi_mean[h]
        save_fname = f"{output_dir}/stats/{result}_pca-{pca_comp}_align-{alignment}_hemi-{hemisphere}_ttest.gii"
        write_gifti_v2(stats.astype(np.float32),
        template_fn = os.path.join(suma_dir, f"{hemilabels[h]}.pial.gii"), 
        output_fn = save_fname)
    print("* saved output (thresholded average r) for both hemispheres")


# compute paired sample t-tests

In [ ]:
from itertools import combinations
results = ['actions-r', 'bg-r',  'agents-r']
pairs = list(combinations(results, 2))

In [ ]:
pairs

In [ ]:

for p1, p2 in pairs:
    hemi_t = []
    hemi_p = []
    hemi_mean = []
    print(f"starting {p1} {p2}________________")
    for hemisphere in hemis:
        medial_mask = np.load(os.path.join(main_dir, 'data', f'fsaverage6_medial_{hemisphere}.npy'))
        assert np.sum(medial_mask) == n_medial[hemisphere]
        cortical_vertices = ~medial_mask # boolean (true for non-medials, false for medials)
        cortical_coords = np.where(cortical_vertices)[0] # returns indices of non-medials
        avg_all = []
        
        # NOTE 1. Fisher z transform per run per participant (np.arctanh)
        for test_subject in subjects:
            stack_fisherz_p1run = []
            stack_fisherz_p2run = []
            # stack_p1run = []
            # stack_p2run = []
            for test_run in runs:           
                p1_data = np.load(f"{output_dir}/{p1}_pca-{pca_comp}_align-{alignment}_{test_subject}_run-{test_run}_hemi-{hemisphere}.npy")
                fisherz_p1run = np.arctanh(p1_data[0, cortical_vertices])
                stack_fisherz_p1run.append(fisherz_p1run)

                p2_data = np.load(f"{output_dir}/{p2}_pca-{pca_comp}_align-{alignment}_{test_subject}_run-{test_run}_hemi-{hemisphere}.npy")
                fisherz_p2run = np.arctanh(p2_data[0, cortical_vertices])
                stack_fisherz_p2run.append(fisherz_p2run)
        
            # NOTE: 2. average (z-transformed) correlations across runs: yields 18 maps (1 per subject)
            # avg_run = np.mean(np.vstack(stack_fisherz_run), axis = 0)
            stack_p1run = np.vstack(stack_fisherz_p1run)
            stack_p2run = np.vstack(stack_fisherz_p2run)
            diff = stack_p1run - stack_p2run
            # print(stack_run.shape)
            # raise
            avg_run = np.mean(diff, axis = 0)
            avg_all.append(avg_run)
        fisherz_all = np.vstack(avg_all) # 18 x nonmedial (per hemi)

        # NOTE: 3. Scipy ttest_1samp to get t-value and p-value
        t, p = scipy.stats.ttest_1samp(fisherz_all, popmean=0,axis=0, alternative='two-sided')
        hemi_t.append(t)
        hemi_p.append(p)
        hemi_mean.append(np.tanh(np.nanmean(fisherz_all, axis = 0)))

    # %% NOTE: 4-1. concatenate (np.hstack) the two hemispheres p-values (and exclude medial wall) prior to computing FDR (load in cortical_vertices.npy)
    left_vert = hemi_p[0].shape[0]
    t_all = np.hstack(hemi_t)
    p_all = np.hstack(hemi_p)

    reject, q_all, _, _ = multipletests(p_all, method = 'fdr_bh')
    q_both = [q_all[:left_vert], q_all[left_vert:]]
    print("* completed t-tests for both hemispheres")
    hemilabels = ['lh', 'rh']
    Path(join(output_dir, 'stats')).mkdir(parents = True, exist_ok = True)
    for h, hemisphere in enumerate(hemis):
        stats = np.zeros((n_vertices))
        hemi_mean[h][q_both[h] >= .05] = 0
        medial_mask = np.load(os.path.join(main_dir, 'data', f'fsaverage6_medial_{hemisphere}.npy'))
        assert np.sum(medial_mask) == n_medial[hemisphere]
        cortical_vertices = ~medial_mask # boolean (true for non-medials, false for medials)
        cortical_coords = np.where(cortical_vertices)[0] # returns indices of non-medials
        stats[cortical_coords] = hemi_mean[h]
        save_fname = f"{output_dir}/stats/{p1[:-2]}-{p2[:-2]}_pca-{pca_comp}_align-{alignment}_hemi-{hemisphere}_ttest.gii"
        write_gifti_v2(stats.astype(np.float32),
        template_fn = os.path.join(suma_dir, f"{hemilabels[h]}.pial.gii"), 
        output_fn = save_fname)
    print("* saved output (thresholded average r) for both hemispheres")


# plot results in fslr

plot t-test results in surfplots, Freesurfer surface plots

In [ ]:
from neuromaps.datasets import fetch_fsaverage, fetch_fslr
import numpy as np
from surfplot import Plot
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib import colors
from os.path import join
import nibabel as nib
surfaces = fetch_fsaverage()
lh, rh = surfaces['inflated']
# main_dir = '/dartfs/rc/lab/D/DBIC/DBIC/f0042x1/life-encoding
main_dir = '/Volumes/life-encoding'
# output_dir = os.path.join(main_dir, 'results', 'himalaya', f"single-{features}", 'ha_common_pca-40')
gii_dir = join(output_dir, 'stats')
print(gii_dir)
alignment = 'ha_common'

In [ ]:
# key = 'bg-r'
# # bgnib = nib.load(join(gii_dir,f'{key}_align-{alignment}_hemi-lh_ttest.gii'))
# bgLfname = f"{gii_dir}/{key}_pca-40_align-{alignment}_hemi-lh_ttest.gii"
# bgRfname = f"{gii_dir}/{key}_pca-40_align-{alignment}_hemi-rh_ttest.gii"
# bgL = nib.load(f"{gii_dir}/{key}_pca-40_align-{alignment}_hemi-lh_ttest.gii")
# bgR = nib.load(f"{gii_dir}/{key}_pca-40_align-{alignment}_hemi-rh_ttest.gii")

In [ ]:
def fsaverage_to_fslr_and_plot(Lfname, Rfname, key, cmap='inferno', max=.20):
    from neuromaps.datasets import fetch_fslr
    from neuromaps.transforms import fsaverage_to_fslr
    giiL = nib.load(Lfname)
    giiR = nib.load(Rfname)
    L_fslr = fsaverage_to_fslr(giiL, target_density='32k', hemi='L', method='linear')
    R_fslr = fsaverage_to_fslr(giiR, target_density='32k', hemi='R', method='linear')
    
    surfaces_fslr = fetch_fslr()
    lh_fslr, rh_fslr = surfaces_fslr['inflated']

    color_range = (0,max)
    p = Plot(surf_lh=lh_fslr, 
             surf_rh=rh_fslr, 
             size=(1000, 200), 
             zoom=1.2, layout='row', 
             views=['lateral', 'medial', 'ventral', 'posterior'], 
             mirror_views=True, brightness=.7)
    p.add_layer({'left': L_fslr[0], 
                'right': R_fslr[0]}, 
                cmap=cmap, cbar=True,
                color_range=color_range,
                cbar_label=key
                ) # YlOrRd_r
    cbar_kws = dict(outer_labels_only=True, pad=.02, n_ticks=2, decimals=3)
    fig = p.build(cbar_kws=cbar_kws)
    # fig.show()
    return(fig)


In [ ]:
gii_dir

In [ ]:
alignment = 'ha_common'
# ['bg', 'agents', 'actions', 'moten']
for feature in ['bg', 'agents', 'actions', 'moten']:
    bgLfname = f"{gii_dir}/{feature}-r_pca-40_align-{alignment}_hemi-lh_ttest.gii"
    bgRfname = f"{gii_dir}/{feature}-r_pca-40_align-{alignment}_hemi-rh_ttest.gii"
    fsaverage_to_fslr_and_plot(bgLfname, bgRfname, feature, cmap='inferno', max=.2)

## count significant voxels
How many voxels are significant per stat map (FDR-corrected)?

In [ ]:
alignment = 'ha_common'
# ['bg', 'agents', 'actions', 'moten']
for feature in ['bg', 'agents', 'actions', 'moten']:
# feature = 'bg'
    Lfname = f"{gii_dir}/{feature}-r_pca-40_align-{alignment}_hemi-lh_ttest.gii"
    Rfname = f"{gii_dir}/{feature}-r_pca-40_align-{alignment}_hemi-rh_ttest.gii"
    # fsaverage_to_fslr_and_plot(bgLfname, bgRfname, feature, cmap='inferno', max=.2)

    giiL = nib.load(Lfname)
    giiR = nib.load(Rfname)

    sig_voxel = np.sum(giiL.agg_data() > 0) + np.sum(giiR.agg_data() > 0) 
    print(f"{feature}: {sig_voxel}")


# cluster thresholded

We threshold the maps using work bench. The output is a (semi) boolean mask, thereby, we apply the mask back onto the t-test maps. 
Steps include
* Workbench cluster threshold
* multiply workbench mask with t-test maps
* plot thresholded maps

### workbench cluster threshold

In [ ]:
from subprocess import call
# parameteres
hemis = ['lh', 'rh']
features = ['bg', 'agents', 'actions', 'moten']
clustersize = 100
hemi_dict = {'lh': 'L', 'rh': 'R'}
fsaverage_dir = '/Users/h/neuromaps-data/atlases/fsaverage'
data_dir = '/Volumes/life-encoding/results/himalaya/glove/ha_common_pca-40/stats'

# thresholding via workbench
for hemi in hemis:
    for feature in features:
        fsaverage_template = f"{fsaverage_dir}/tpl-fsaverage_den-41k_hemi-{hemi_dict[hemi]}_inflated.surf.gii"
        himalaya = f"{data_dir}/{feature}-r_pca-40_align-ha_common_hemi-{hemi}_ttest.gii"
        outputfname = f"{data_dir}/{feature}-r_pca-40_align-ha_common_hemi-{hemi}_ttest_cluster-{clustersize}.gii"
        return_code = call(f"wb_command -metric-find-clusters {fsaverage_template} {himalaya} 0 {clustersize} {outputfname}", shell=True)
print(f"Return: {return_code}")

## check cluster maps
The images will look like masks

In [ ]:
for feature in ['bg', 'agents', 'actions', 'moten']:
    clusterthresL = f"{data_dir}/{feature}-r_pca-40_align-ha_common_hemi-lh_ttest_cluster-{clustersize}.gii"
    clusterthresR = f"{data_dir}/{feature}-r_pca-40_align-ha_common_hemi-rh_ttest_cluster-{clustersize}.gii"
    fsaverage_to_fslr_and_plot(clusterthresL, clusterthresR, f'{feature}-r', cmap = 'Reds')

## multiply workbench cluster threshold maps to t-test maps

In [ ]:
clustersize

In [ ]:
def write_gifti_v2(data, output_fn, template_fn):
    gii = nib.load(template_fn)
    for i in np.arange(gii.numDA):
        gii.remove_gifti_data_array(0)
    gda = nib.gifti.GiftiDataArray(data)
    gii.add_gifti_data_array(gda)
    # nib.gifti.write(gii, output_fn)
    gii.to_filename(output_fn)

hemis = ['lh', 'rh']
hemi_dict = {'lh': 'L', 'rh': 'R'}
features = ['bg', 'agents', 'actions', 'moten']
# clustersize = 50

fsaverage_dir = '/Users/h/neuromaps-data/atlases/fsaverage'
data_dir = '/Volumes/life-encoding/results/himalaya/glove/ha_common_pca-40/stats'

# thresholding via workbench
for hemi in hemis:
    for feature in features:
        clustermask_fname = f"{data_dir}/{feature}-r_pca-40_align-ha_common_hemi-{hemi}_ttest_cluster-{clustersize}.gii"
        clustL = nib.load(clustermask_fname).agg_data() > 0
        himalaya_fname = f"{gii_dir}/{feature}-r_pca-40_align-ha_common_hemi-{hemi}_ttest.gii"
        himalaya = nib.load(himalaya_fname).agg_data()
        maskeddata = clustL * himalaya
        write_gifti_v2(maskeddata.astype(np.float32), 
            output_fn=f"{data_dir}/{feature}-r_pca-40_align-{alignment}_hemi-{hemi}_ttest_thres-cluster{clustersize}.gii",
            template_fn=f"{fsaverage_dir}/tpl-fsaverage_den-41k_hemi-{hemi_dict[hemi]}_pial.surf.gii")

## plot thresholded maps in fslr [manuscript]

In [ ]:
main_dir

In [ ]:
save_dir = '/Users/h/Documents/projects_local/life-encoding/figure'
for feature in ['bg', 'agents', 'actions', 'moten']:
    thresL = f"{data_dir}/{feature}-r_pca-40_align-{alignment}_hemi-lh_ttest_thres-cluster{clustersize}.gii"
    thresR = f"{data_dir}/{feature}-r_pca-40_align-{alignment}_hemi-rh_ttest_thres-cluster{clustersize}.gii"
    fig = fsaverage_to_fslr_and_plot(thresL, thresR, f'{feature}-r', cmap = 'inferno', max=.2)
    fig.savefig(join(save_dir, f"{feature}-r_pca-40_align-{alignment}_hemi-lh_ttest_thres-cluster{clustersize}.png"), dpi=300)

# contrast maps

In [ ]:
from itertools import combinations
results = ['actions-r', 'bg-r',  'agents-r']
pairs = list(combinations(results, 2))

In [ ]:
pca_num = 40
for p1, p2 in pairs:
    hemi_t = []
    hemi_p = []
    hemi_mean = []
    print(f"starting {p1} {p2}________________")
    for hemisphere in hemis:
        medial_mask = np.load(os.path.join(main_dir, 'data', f'fsaverage6_medial_{hemisphere}.npy'))
        assert np.sum(medial_mask) == n_medial[hemisphere]
        cortical_vertices = ~medial_mask # boolean (true for non-medials, false for medials)
        cortical_coords = np.where(cortical_vertices)[0] # returns indices of non-medials
        avg_all = []
        
        # NOTE 1. Fisher z transform per run per participant (np.arctanh)
        for test_subject in subjects:
            stack_fisherz_p1run = []
            stack_fisherz_p2run = []
            # stack_p1run = []
            # stack_p2run = []
            for test_run in runs:           
                p1_data = np.load(f"{output_dir}/{p1}_pca-{pca_num}_align-{alignment}_{test_subject}_run-{test_run}_hemi-{hemisphere}.npy")
                fisherz_p1run = np.arctanh(p1_data[0, cortical_vertices])
                stack_fisherz_p1run.append(fisherz_p1run)

                p2_data = np.load(f"{output_dir}/{p2}_pca-{pca_num}_align-{alignment}_{test_subject}_run-{test_run}_hemi-{hemisphere}.npy")
                fisherz_p2run = np.arctanh(p2_data[0, cortical_vertices])
                stack_fisherz_p2run.append(fisherz_p2run)
        
            # NOTE: 2. average (z-transformed) correlations across runs: yields 18 maps (1 per subject)
            # avg_run = np.mean(np.vstack(stack_fisherz_run), axis = 0)
            stack_p1run = np.vstack(stack_fisherz_p1run)
            stack_p2run = np.vstack(stack_fisherz_p2run)
            diff = stack_p1run - stack_p2run
            # print(stack_run.shape)
            # raise
            avg_run = np.mean(diff, axis = 0)
            avg_all.append(avg_run)
        fisherz_all = np.vstack(avg_all) # 18 x nonmedial (per hemi)

        # NOTE: 3. Scipy ttest_1samp to get t-value and p-value
        t, p = scipy.stats.ttest_1samp(fisherz_all, popmean=0,axis=0, alternative='two-sided')
        hemi_t.append(t)
        hemi_p.append(p)
        hemi_mean.append(np.tanh(np.nanmean(fisherz_all, axis = 0)))

    # %% NOTE: 4-1. concatenate (np.hstack) the two hemispheres p-values (and exclude medial wall) prior to computing FDR (load in cortical_vertices.npy)
    left_vert = hemi_p[0].shape[0]
    t_all = np.hstack(hemi_t)
    p_all = np.hstack(hemi_p)

    reject, q_all, _, _ = multipletests(p_all, method = 'fdr_bh')
    q_both = [q_all[:left_vert], q_all[left_vert:]]
    print("* completed t-tests for both hemispheres")
    hemilabels = ['lh', 'rh']
    Path(join(output_dir, 'stats')).mkdir(parents = True, exist_ok = True)
    for h, hemisphere in enumerate(hemis):
        stats = np.zeros((n_vertices))
        hemi_mean[h][q_both[h] >= .05] = 0
        medial_mask = np.load(os.path.join(main_dir, 'data', f'fsaverage6_medial_{hemisphere}.npy'))
        assert np.sum(medial_mask) == n_medial[hemisphere]
        cortical_vertices = ~medial_mask # boolean (true for non-medials, false for medials)
        cortical_coords = np.where(cortical_vertices)[0] # returns indices of non-medials
        stats[cortical_coords] = hemi_mean[h]
        save_fname = f"{output_dir}/stats/{p1[:-2]}-{p2[:-2]}_pca-{pca_num}_align-{alignment}_hemi-{hemisphere}_ttest.gii"
        write_gifti_v2(stats.astype(np.float32),
        template_fn = os.path.join(suma_dir, f"{hemilabels[h]}.pial.gii"), 
        output_fn = save_fname)
    print("* saved output (thresholded average r) for both hemispheres")


## cluster correct contrast maps

In [ ]:
def fsaverage_to_fslr_contrast(Lfname, Rfname, key, cmap='inferno', max=.20):
    from neuromaps.datasets import fetch_fslr
    from neuromaps.transforms import fsaverage_to_fslr
    giiL = nib.load(Lfname)
    giiR = nib.load(Rfname)
    L_fslr = fsaverage_to_fslr(giiL, target_density='32k', hemi='L', method='linear')
    R_fslr = fsaverage_to_fslr(giiR, target_density='32k', hemi='R', method='linear')
    
    surfaces_fslr = fetch_fslr()
    lh_fslr, rh_fslr = surfaces_fslr['inflated']

    color_range = (-1*max,max)
    p = Plot(surf_lh=lh_fslr, 
             surf_rh=rh_fslr, 
             size=(1000, 200), 
             zoom=1.2, layout='row', 
             views=['lateral', 'medial', 'ventral', 'posterior'], 
             mirror_views=True, brightness=.7)
    p.add_layer({'left': L_fslr[0], 
                'right': R_fslr[0]}, 
                cmap=cmap, cbar=True,
                color_range=color_range,
                cbar_label=key
                ) # YlOrRd_r
    cbar_kws = dict(outer_labels_only=True, pad=.02, n_ticks=2, decimals=3)
    fig = p.build(cbar_kws=cbar_kws)
    # fig.show()
    return(fig)


In [ ]:
gii_dir = join(output_dir, 'stats')
gii_dir

In [ ]:
from subprocess import call
# parameteres
hemis = ['lh', 'rh']
clustersize = 100
hemi_dict = {'lh': 'L', 'rh': 'R'}
fsaverage_dir = '/Users/h/neuromaps-data/atlases/fsaverage'
data_dir = '/Volumes/life-encoding/results/himalaya/glove/ha_common_pca-40/stats'

# thresholding via workbench
for hemi in hemis:
    # for feature in features:
    for key1, key2 in pairs:
        fsaverage_template = f"{fsaverage_dir}/tpl-fsaverage_den-41k_hemi-{hemi_dict[hemi]}_inflated.surf.gii"
        himalaya = f"{data_dir}/{key1[:-2]}-{key2[:-2]}_pca-40_align-ha_common_hemi-{hemi}_ttest.gii"
        outputfname = f"{data_dir}/{key1[:-2]}-{key2[:-2]}_pca-40_align-ha_common_hemi-{hemi}_ttest_cluster-{clustersize}.gii"
        return_code = call(f"wb_command -metric-find-clusters {fsaverage_template} {himalaya} 0 {clustersize} {outputfname}", shell=True)
print(f"Return: {return_code}")

In [ ]:
from nilearn.plotting import plot_surf
from nilearn.plotting import plot_surf_stat_map
import matplotlib.pyplot as plt
from nilearn import datasets
from matplotlib import colors
fsaverage = datasets.fetch_surf_fsaverage(mesh = 'fsaverage6')
gii_dir = join(output_dir, 'stats')


### multiply cluster-thresholded boolean mask and data

In [ ]:

for hemi in hemis:
    for key1, key2 in pairs:
        clustermask_fname = f"{data_dir}/{key1[:-2]}-{key2[:-2]}_pca-40_align-ha_common_hemi-{hemi}_ttest_cluster-{clustersize}.gii"
        clustL = nib.load(clustermask_fname).agg_data() > 0
        himalaya_fname = f"{gii_dir}/{key1[:-2]}-{key2[:-2]}_pca-40_align-ha_common_hemi-{hemi}_ttest.gii"
        himalaya = nib.load(himalaya_fname).agg_data()
        maskeddata = clustL * himalaya
        write_gifti_v2(maskeddata.astype(np.float32), 
            output_fn=f"{data_dir}/{key1[:-2]}-{key2[:-2]}_pca-40_align-{alignment}_hemi-{hemi}_ttest_thres-cluster{clustersize}.gii",
            template_fn=f"{fsaverage_dir}/tpl-fsaverage_den-41k_hemi-{hemi_dict[hemi]}_pial.surf.gii")

In [ ]:
key1 = 'actions'; key2 =  'bg'
contrastL = join(data_dir, f'{key1}-{key2}_pca-40_align-{alignment}_hemi-lh_ttest_thres-cluster{clustersize}.gii')
contrastR = join(data_dir, f'{key1}-{key2}_pca-40_align-{alignment}_hemi-rh_ttest_thres-cluster{clustersize}.gii')
fig = fsaverage_to_fslr_contrast(contrastL, contrastR, f"{key1}_{key2}", cmap = 'coolwarm', max=.2)

In [ ]:
fig = fsaverage_to_fslr_contrast(contrastL, contrastR, f"{key1}_{key2}", cmap = 'coolwarm', max=.2)

In [ ]:
key1 = 'actions'; key2 =  'agents'
contrastL = join(data_dir, f'{key1}-{key2}_pca-40_align-{alignment}_hemi-lh_ttest_thres-cluster{clustersize}.gii')
contrastR = join(data_dir, f'{key1}-{key2}_pca-40_align-{alignment}_hemi-rh_ttest_thres-cluster{clustersize}.gii')
fig = fsaverage_to_fslr_contrast(contrastL, contrastR, f"{key1}_{key2}", cmap = 'inferno', max=.2)

In [ ]:
# actions-agents
# actions-bg_
# bg-agents

key1 = 'bg'; key2 =  'agents'
contrastL = join(data_dir, f'{key1}-{key2}_pca-40_align-{alignment}_hemi-lh_ttest_thres-cluster{clustersize}.gii')
contrastR = join(data_dir, f'{key1}-{key2}_pca-40_align-{alignment}_hemi-rh_ttest_thres-cluster{clustersize}.gii')
fig = fsaverage_to_fslr_contrast(contrastL, contrastR, f"{key1}_{key2}", cmap = 'coolwarm', max=.2)